#Predicting Olympic medals with Deep Learning

- It's almost Olympics time, and we're here to predict medals!
- Let's start out by installing and importing everything we need:

In [5]:
!pip install optuna  > /dev/null 2>&1

In [6]:
!pip install pandas numpy scikit-learn torch transformers  > /dev/null 2>&1

In [7]:
!pip install wikipedia-api > /dev/null 2>&1

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_squared_error

import torch
import torch.nn as nn
from transformers import pipeline
from torch.utils.data import DataLoader, TensorDataset

import datetime
import wikipediaapi

If you have a GPU, this is your best friend:

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


Let's hard-code the participating countries. Each country has a 3-letter code called its NOC. It will be super handy today.

In [131]:
# List of tuples mapping countries to their NOCs
participating_countries = [
    # Africa
    ("Algeria", "ALG"),
    ("Angola", "ANG"),
    ("Benin", "BEN"),
    ("Botswana", "BOT"),
    ("Burkina Faso", "BUR"),
    ("Burundi", "BDI"),
    ("Cameroon", "CMR"),
    ("Cabo Verde", "CPV"),
    ("Central African Republic", "CAF"),
    ("Chad", "CHA"),
    ("Comoros", "COM"),
    ("Congo", "CGO"),
    ("Democratic Republic of the Congo", "COD"),
    ("Côte d’Ivoire", "CIV"),
    ("Djibouti", "DJI"),
    ("Egypt", "EGY"),
    ("Eritrea", "ERI"),
    ("Eswatini", "SWZ"),
    ("Ethiopia", "ETH"),
    ("Gabon", "GAB"),
    ("Gambia", "GAM"),
    ("Ghana", "GHA"),
    ("Guinea", "GUI"),
    ("Guinea-Bissau", "GBS"),
    ("Equatorial Guinea", "GEQ"),
    ("Kenya", "KEN"),
    ("Lesotho", "LES"),
    ("Liberia", "LBR"),
    ("Libya", "LBA"),
    ("Madagascar", "MAD"),
    ("Malawi", "MAW"),
    ("Mali", "MLI"),
    ("Morocco", "MAR"),
    ("Mauritius", "MRI"),
    ("Mauritania", "MTN"),
    ("Mozambique", "MOZ"),
    ("Namibia", "NAM"),
    ("Niger", "NIG"),
    ("Nigeria", "NGR"),
    ("Uganda", "UGA"),
    ("Rwanda", "RWA"),
    ("Sao Tome and Principe", "STP"),
    ("Senegal", "SEN"),
    ("Seychelles", "SEY"),
    ("Sierra Leone", "SLE"),
    ("Somalia", "SOM"),
    ("South Africa", "RSA"),
    ("South Sudan", "SSD"),
    ("Sudan", "SUD"),
    ("United Republic of Tanzania", "TAN"),
    ("Togo", "TOG"),
    ("Tunisia", "TUN"),
    ("Zambia", "ZAM"),
    ("Zimbabwe", "ZIM"),

    # The Americas
    ("Antigua and Barbuda", "ANT"),
    ("Argentina", "ARG"),
    ("Aruba", "ARU"),
    ("Bahamas", "BAH"),
    ("Barbados", "BAR"),
    ("Belize", "BIZ"),
    ("Bermuda", "BER"),
    ("Bolivia", "BOL"),
    ("Brazil", "BRA"),
    ("Cayman Islands", "CAY"),
    ("Canada", "CAN"),
    ("Chile", "CHI"),
    ("Colombia", "COL"),
    ("Costa Rica", "CRC"),
    ("Cuba", "CUB"),
    ("Dominican Republic", "DOM"),
    ("Dominica", "DMA"),
    ("El Salvador", "ESA"),
    ("Ecuador", "ECU"),
    ("Grenada", "GRN"),
    ("Guatemala", "GUA"),
    ("Guyana", "GUY"),
    ("Haiti", "HAI"),
    ("Honduras", "HON"),
    ("Jamaica", "JAM"),
    ("Mexico", "MEX"),
    ("Nicaragua", "NCA"),
    ("Panama", "PAN"),
    ("Paraguay", "PAR"),
    ("Peru", "PER"),
    ("Puerto Rico", "PUR"),
    ("Saint Kitts and Nevis", "SKN"),
    ("Saint Lucia", "LCA"),
    ("St. Vincent and the Grenadines", "VIN"),
    ("Suriname", "SUR"),
    ("Trinidad and Tobago", "TTO"),
    ("United States", "USA"),
    ("Uruguay", "URU"),
    ("Venezuela", "VEN"),
    ("Virgin Islands, British", "IVB"),
    ("United States Virgin Islands", "ISV"),

    # Asia
    ("Afghanistan", "AFG"),
    ("Bahrain", "BRN"),
    ("Bangladesh", "BAN"),
    ("Bhutan", "BHU"),
    ("Brunei Darussalam", "BRU"),
    ("Cambodia", "CAM"),
    ("China", "CHN"),
    ("Republic of Korea", "KOR"),
    ("Hong Kong, China", "HKG"),
    ("India", "IND"),
    ("Indonesia", "INA"),
    ("Islamic Republic of Iran", "IRI"),
    ("Iraq", "IRQ"),
    ("Japan", "JPN"),
    ("Jordan", "JOR"),
    ("Kazakhstan", "KAZ"),
    ("Kyrgyzstan", "KGZ"),
    ("Kuwait", "KUW"),
    ("Lao People’s Democratic Republic", "LAO"),
    ("Lebanon", "LBN"),
    ("Malaysia", "MAS"),
    ("Maldives", "MDV"),
    ("Mongolia", "MGL"),
    ("Myanmar", "MYA"),
    ("Nepal", "NEP"),
    ("Oman", "OMA"),
    ("Pakistan", "PAK"),
    ("Palestine", "PLE"),
    ("Philippines", "PHI"),
    ("Qatar", "QAT"),
    ("Democratic People’s Republic of Korea", "PRK"),
    ("Saudi Arabia", "KSA"),
    ("Singapore", "SGP"),
    ("Sri Lanka", "SRI"),
    ("Syrian Arab Republic", "SYR"),
    ("Tajikistan", "TJK"),
    ("Chinese Taipei", "TPE"),
    ("Thailand", "THA"),
    ("East Timor", "TLS"),
    ("Turkmenistan", "TKM"),
    ("United Arab Emirates", "UAE"),
    ("Uzbekistan", "UZB"),
    ("Vietnam", "VIE"),
    ("Yemen", "YEM"),

    # Europe
    ("Albania", "ALB"),
    ("Andorra", "AND"),
    ("Armenia", "ARM"),
    ("Austria", "AUT"),
    ("Azerbaijan", "AZE"),
    ("Belgium", "BEL"),
    ("Bosnia and Herzegovina", "BIH"),
    ("Bulgaria", "BUL"),
    ("Cyprus", "CYP"),
    ("Croatia", "CRO"),
    ("Czechia", "CZE"),
    ("Denmark", "DEN"),
    ("Spain", "ESP"),
    ("Estonia", "EST"),
    ("Finland", "FIN"),
    ("France", "FRA"),
    ("Georgia", "GEO"),
    ("Germany", "GER"),
    ("Great Britain", "GBR"),
    ("Greece", "GRE"),
    ("Hungary", "HUN"),
    ("Ireland", "IRL"),
    ("Iceland", "ISL"),
    ("Israel", "ISR"),
    ("Italy", "ITA"),
    ("Kosovo", "KOS"),
    ("Latvia", "LAT"),
    ("Liechtenstein", "LIE"),
    ("Lithuania", "LTU"),
    ("Luxembourg", "LIE"),
    ("North Macedonia", "MKD"),
    ("Malta", "MLT"),
    ("Republic of Moldova", "MDA"),
    ("Monaco", "MON"),
    ("Montenegro", "MNE"),
    ("Netherlands", "NED"),
    ("Norway", "NOR"),
    ("Poland", "POL"),
    ("Portugal", "POR"),
    ("Romania", "ROU"),
    ("San Marino", "SMR"),
    ("Serbia", "SRB"),
    ("Slovakia", "SVK"),
    ("Slovenia", "SLO"),
    ("Sweden", "SWE"),
    ("Switzerland", "SUI"),
    ("Türkiye", "TUR"),
    ("Ukraine", "UKR"),

    # Oceania
    ("American Samoa", "ASA"),
    ("Australia", "AUS"),
    ("Cook Islands", "COK"),
    ("Fiji", "FIJ"),
    ("Guam", "GUM"),
    ("Kiribati", "KIR"),
    ("Marshall Islands", "MHL"),
    ("Federated States of Micronesia", "FSM"),
    ("Nauru", "NRU"),
    ("New Zealand", "NZL"),
    ("Palau", "PLW"),
    ("Papua New Guinea", "PNG"),
    ("Solomon Islands", "SOL"),
    ("Samoa", "SAM"),
    ("Tonga", "TGA"),
    ("Tuvalu", "TUV"),
    ("Vanuatu", "VAN")
]


Sorry *insert name of your elementary school teacher here*, we'll be using Wikipedia!

In [11]:
#Initialize Wikipedia API
wiki_wiki = wikipediaapi.Wikipedia(user_agent='My_Olympic_Data_Bot/1.0 (randomemail@gmail.com)')

#Initialize transformer pipeline for text feature extraction
nlp = pipeline("feature-extraction", model="bert-base-uncased", device=device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Transformers are super cool. Let's see what a transformer can fetch and tell us about different countries from Wikipedia.

In [133]:
# Function to extract text features from Wikipedia
def get_country_features(country_name):
    page = wiki_wiki.page(country_name)

    if not page.exists():
        print(f"Wikipedia page not found for: {country_name}")
        return None

    # Extract summary text
    summary_text = page.summary

    # Truncate the summary text to fit within the BERT model's limit
    # BERT models typically have a maximum sequence length of 512 tokens
    tokens = summary_text.split()
    tokens = tokens[:512] # Truncate to 512 tokens

    # Use the transformer model to extract features
    features = nlp(summary_text, truncation=True, max_length=512 )

    # Aggregating features (e.g., mean across the tokens)
    aggregated_features = [sum(x)/len(x) for x in zip(*features[0])]

    return aggregated_features

In [134]:
# Function to compile features for all countries
def compile_country_features():
    all_country_features = {}

    for country_name, NOC in participating_countries:
        features = get_country_features(country_name)
        if features:
            all_country_features[NOC] = features

    return all_country_features

# Compile features for all countries
country_features = compile_country_features()

# Display sample features
print(f"Sample features for United States: {country_features.get('United States')}")

Sample features for United States: None


This is why we need NOC's. So we can use our data easily and in a uniform way. Look:

U S A! U S A!

In [135]:
print(f"Sample features for United States: {country_features.get('USA')}")

Sample features for United States: [-0.4611632887737187, -0.05057181595475413, 0.04607897073651657, 0.03584708038806639, 0.07485528242796136, -0.037889812905973486, 0.32460331383856555, 0.6609784091178881, 0.016430025981662766, -0.03134424225658705, 0.01804722530619074, -0.3637100045598345, -0.3478598260585386, 0.746809978840929, -0.13590981190054663, 0.36191020858905176, 0.39534079708084846, 0.08246361629213084, -0.31315066141041825, 0.58237973106543, 0.24851076140134865, 0.1570819512462549, 0.01223808385054781, 0.8492528961123753, 0.6276254809317834, -0.027949743085798673, -0.1848762339005816, -0.27075870790940826, -0.08797866366830931, -0.1051676446016927, 0.512074131146619, -0.2959192017474379, -0.27408951300367335, -0.17718606423136407, 0.2642809900868315, 0.0018256630521022998, -0.08326344722354406, -0.24185672438306938, 0.03404435755987478, 0.18204859425156883, -0.6288830170960864, -0.41536303806148567, 0.010657959245286008, 0.10857912525960955, 0.006136733687640117, -0.09405646

You should be getting long vectors of numbers, which are our bert-generated features. If you're getting None here, something is wrong.

In [136]:
print(f"Sample features for China: {country_features.get('CHN')}")

Sample features for China: [-0.4225372159318539, 0.07738997079513865, -0.00876026387300044, -0.0453335340087051, 0.0705637495798328, 0.06537318900478795, 0.20705597874439263, 0.8367145173560857, 0.07901886195443808, -0.027523922080376906, 0.0548064801831174, -0.41264560758099833, -0.3421829218887069, 0.5686385280141977, -0.1718181520982398, 0.48052678947721006, 0.3179122218880366, 0.05206499707946932, -0.2997113193325731, 0.600626012282305, 0.17527564164268838, 0.06800104731212286, 0.035996212052737064, 0.9558984380591937, 0.592128022322413, 0.006404183533845753, -0.07678748904572785, 0.016648983480337165, -0.11963628074613553, -0.2725862703767632, 0.400021027551702, -0.11248992903676935, -0.3268878680675016, -0.20129200645212109, 0.12896272074476656, 0.05852384559238999, -0.023480092269551278, -0.11404357923265707, -0.0850943872908374, 0.08804373230987039, -0.7671799780866877, -0.3456515371608475, -0.01934311230934327, 0.28416899558988007, 0.12581931660329815, -0.20934594558373476, 0.

Ok, now let's look at some data and start preprocessing it. Paris2024 is a Summer Olympics, so we'll only be looking at Summer editions.

In [143]:
historical_medal_tally_path = 'Olympic_Games_Medal_Tally.csv'
paris_2024_medal_tally_path = 'medals_total.csv'

# Load the CSV files into dataframes
historical_medal_tally = pd.read_csv(historical_medal_tally_path)
historical_medal_tally.rename(columns={'country_noc': 'country_code'}, inplace=True)

# Filter the data for Summer Olympics only
historical_medal_tally['edition'] = historical_medal_tally['edition'].str.replace(' Olympics', '')
historical_medal_tally[['Year', 'Season']] = historical_medal_tally['edition'].str.split(' ', expand=True)
historical_medal_tally = historical_medal_tally.drop(columns=['year'])
historical_medal_tally_summer = historical_medal_tally[historical_medal_tally['Season'] == 'Summer']
paris_2024_medal_tally = pd.read_csv(paris_2024_medal_tally_path)

participating_countries_df = pd.DataFrame(participating_countries, columns=['country', 'country_code'])


historical_medal_tally_summer


,edition,edition_id,country,country_code,gold,silver,bronze,total,Year,Season
0,1896 Summer,1,United States,USA,11,7,2,20,1896,Summer
1,1896 Summer,1,Greece,GRE,10,18,19,47,1896,Summer
2,1896 Summer,1,Germany,GER,6,5,2,13,1896,Summer
3,1896 Summer,1,France,FRA,5,4,2,11,1896,Summer
4,1896 Summer,1,Great Britain,GBR,2,3,2,7,1896,Summer
...,...,...,...,...,...,...,...,...,...,...
1338,2020 Summer,61,Ghana,GHA,0,0,1,1,2020,Summer
1339,2020 Summer,61,Grenada,GRN,0,0,1,1,2020,Summer
1340,2020 Summer,61,Kuwait,KUW,0,0,1,1,2020,Summer
1341,2020 Summer,61,Republic of Moldova,MDA,0,0,1,1,2020,Summer


Not all countries have won medals yet, and yet we can't completely disregard them- there might be surprises!

In [138]:
# Function to add missing countries with 0 medals
def add_missing_countries(medal_tally_df, left_on, right_on):
    # Merge the medal tally with participating countries
    merged_df = pd.merge(participating_countries_df, medal_tally_df, left_on=left_on, right_on=right_on, how='left')

    # Fill NaN values (for countries with no medals) with 0
    merged_df.fillna(-1, inplace=True)

    return merged_df

We'll try two test cases: First we'll filter out countries which haven't won medals recently. This is mainly in order to disregard former countries like East Germany, Soviet Russia and such, which were IMMENSE Olympic powerhouses but no longer exist. We don't want them biasing our data...

In [144]:
# Test case 1: country filtering - predicting only for countries which have won medals in recent Olympics
historical_medal_tally = add_missing_countries(historical_medal_tally_summer, ['country_code'], ['country_code'])
paris_2024_medal_tally = add_missing_countries(paris_2024_medal_tally, ['country_code'], ['country_code'])
historical_medal_tally = historical_medal_tally.drop(columns=['country_y']).rename(columns={'country_x': 'Country'})


historical_medal_tally['Year'] = historical_medal_tally['Year'].astype(int) # Convert 'year' column to integer type
historical_medal_tally

,Country,country_code,edition,edition_id,gold,silver,bronze,total,Year,Season
0,Algeria,ALG,1984 Summer,21.0,0.0,0.0,2.0,2.0,1984,Summer
1,Algeria,ALG,1992 Summer,23.0,1.0,0.0,1.0,2.0,1992,Summer
2,Algeria,ALG,1996 Summer,24.0,2.0,0.0,1.0,3.0,1996,Summer
3,Algeria,ALG,2000 Summer,25.0,1.0,1.0,3.0,5.0,2000,Summer
4,Algeria,ALG,2008 Summer,53.0,0.0,1.0,1.0,2.0,2008,Summer
...,...,...,...,...,...,...,...,...,...,...
1322,Solomon Islands,SOL,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
1323,Samoa,SAM,2008 Summer,53.0,0.0,1.0,0.0,1.0,2008,Summer
1324,Tonga,TGA,1996 Summer,24.0,0.0,1.0,0.0,1.0,1996,Summer
1325,Tuvalu,TUV,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1


Our second case doesn't do that, but rather looks at all the countries participating in Paris2024.

In [145]:
## Test case 2: without country filtering - predicting for all participating countries
historical_medal_tally_case_2 = historical_medal_tally
paris_2024_medal_tally_case_2 = paris_2024_medal_tally
historical_medal_tally_case_2

,Country,country_code,edition,edition_id,gold,silver,bronze,total,Year,Season
0,Algeria,ALG,1984 Summer,21.0,0.0,0.0,2.0,2.0,1984,Summer
1,Algeria,ALG,1992 Summer,23.0,1.0,0.0,1.0,2.0,1992,Summer
2,Algeria,ALG,1996 Summer,24.0,2.0,0.0,1.0,3.0,1996,Summer
3,Algeria,ALG,2000 Summer,25.0,1.0,1.0,3.0,5.0,2000,Summer
4,Algeria,ALG,2008 Summer,53.0,0.0,1.0,1.0,2.0,2008,Summer
...,...,...,...,...,...,...,...,...,...,...
1322,Solomon Islands,SOL,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1
1323,Samoa,SAM,2008 Summer,53.0,0.0,1.0,0.0,1.0,2008,Summer
1324,Tonga,TGA,1996 Summer,24.0,0.0,1.0,0.0,1.0,1996,Summer
1325,Tuvalu,TUV,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1


In [146]:

# Test case 1: Keep only countries which have won at least 8 medals since Sydney 2000 for relevance
historical_medal_tally_filtered = historical_medal_tally[historical_medal_tally['Year'] >= 2000]
medals_by_country = historical_medal_tally_filtered.groupby('country_code')['total'].sum()
countries_with_at_least_8_medals = medals_by_country[medals_by_country >= 8].index
countries_with_at_least_8_medals_df = pd.DataFrame(countries_with_at_least_8_medals)
medals = pd.DataFrame(medals_by_country)
historical_medal_tally_refined = historical_medal_tally[historical_medal_tally['country_code'].isin(countries_with_at_least_8_medals)]
paris_2024_medal_tally_refined = paris_2024_medal_tally[paris_2024_medal_tally['country_code'].isin(countries_with_at_least_8_medals)]
historical_medal_tally_df = pd.DataFrame(historical_medal_tally_refined)
paris_2024_medal_tally_df = pd.DataFrame(paris_2024_medal_tally_refined)

#Test case 2:
historical_medal_tally_case_2_df = pd.DataFrame(historical_medal_tally_case_2)
paris_2024_medal_tally_case_2_df = pd.DataFrame(paris_2024_medal_tally_case_2)

Here are the countries which won medals recently(at least 8 since Sydney2000)

In [147]:
countries_with_at_least_8_medals_df

,country_code
0,ALG
1,ARG
2,ARM
3,AUS
4,AUT
...,...
65,TUR
66,UKR
67,USA
68,UZB


Feel free to view the dataframes

In [148]:
historical_medal_tally_df

,Country,country_code,edition,edition_id,gold,silver,bronze,total,Year,Season
0,Algeria,ALG,1984 Summer,21.0,0.0,0.0,2.0,2.0,1984,Summer
1,Algeria,ALG,1992 Summer,23.0,1.0,0.0,1.0,2.0,1992,Summer
2,Algeria,ALG,1996 Summer,24.0,2.0,0.0,1.0,3.0,1996,Summer
3,Algeria,ALG,2000 Summer,25.0,1.0,1.0,3.0,5.0,2000,Summer
4,Algeria,ALG,2008 Summer,53.0,0.0,1.0,1.0,2.0,2008,Summer
...,...,...,...,...,...,...,...,...,...,...
1315,New Zealand,NZL,2004 Summer,26.0,3.0,2.0,0.0,5.0,2004,Summer
1316,New Zealand,NZL,2008 Summer,53.0,3.0,2.0,4.0,9.0,2008,Summer
1317,New Zealand,NZL,2012 Summer,54.0,6.0,2.0,5.0,13.0,2012,Summer
1318,New Zealand,NZL,2016 Summer,59.0,4.0,9.0,5.0,18.0,2016,Summer


In [149]:
paris_2024_medal_tally_df

,country,country_code,Gold Medal,Silver Medal,Bronze Medal,Total
0,Algeria,ALG,2.0,0.0,1.0,3.0
15,Egypt,EGY,1.0,1.0,1.0,3.0
18,Ethiopia,ETH,1.0,3.0,0.0,4.0
25,Kenya,KEN,4.0,2.0,5.0,11.0
32,Morocco,MAR,1.0,0.0,1.0,2.0
...,...,...,...,...,...,...
184,Switzerland,SUI,1.0,2.0,5.0,8.0
185,Türkiye,TUR,0.0,3.0,5.0,8.0
186,Ukraine,UKR,3.0,5.0,4.0,12.0
188,Australia,AUS,18.0,19.0,16.0,53.0


We promise, this is as much data processing as we'll be doing today. Pretty straightforward

In [154]:
# Preprocessing steps to standardize country codes and names for merging

# Standardize column names for easier merging
historical_medal_tally_df.rename(columns={'total': 'Total', 'gold': 'Gold', 'silver': 'Silver', 'bronze': 'Bronze'}, inplace=True)
historical_medal_tally_case_2_df.rename(columns={'total': 'Total', 'gold': 'Gold', 'silver': 'Silver', 'bronze': 'Bronze'}, inplace=True)
paris_2024_medal_tally_df.rename(columns={'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze'}, inplace=True)
paris_2024_medal_tally_case_2_df.rename(columns={'Gold Medal': 'Gold', 'Silver Medal': 'Silver', 'Bronze Medal': 'Bronze'}, inplace=True)
historical_medal_tally_df



,Country,country_code,edition,edition_id,Gold,Silver,Bronze,Total,Year,Season
0,Algeria,ALG,1984 Summer,21.0,0.0,0.0,2.0,2.0,1984,Summer
1,Algeria,ALG,1992 Summer,23.0,1.0,0.0,1.0,2.0,1992,Summer
2,Algeria,ALG,1996 Summer,24.0,2.0,0.0,1.0,3.0,1996,Summer
3,Algeria,ALG,2000 Summer,25.0,1.0,1.0,3.0,5.0,2000,Summer
4,Algeria,ALG,2008 Summer,53.0,0.0,1.0,1.0,2.0,2008,Summer
...,...,...,...,...,...,...,...,...,...,...
1315,New Zealand,NZL,2004 Summer,26.0,3.0,2.0,0.0,5.0,2004,Summer
1316,New Zealand,NZL,2008 Summer,53.0,3.0,2.0,4.0,9.0,2008,Summer
1317,New Zealand,NZL,2012 Summer,54.0,6.0,2.0,5.0,13.0,2012,Summer
1318,New Zealand,NZL,2016 Summer,59.0,4.0,9.0,5.0,18.0,2016,Summer


Let's only keep the NOC, year and medal tallies. EVerything else isn't so important.

In [155]:
# Extract relevant columns for merging: ['country_code', 'Total']
historical_medal_tally_relevant = historical_medal_tally_df[['country_code', 'Year', 'Total', 'Gold', 'Silver', 'Bronze']]
historical_medal_tally_relevant_case_2 = historical_medal_tally_case_2_df[['country_code', 'Year', 'Total', 'Gold', 'Silver', 'Bronze']]

In [156]:
historical_medal_tally_relevant

,country_code,Year,Total,Gold,Silver,Bronze
0,ALG,1984,2.0,0.0,0.0,2.0
1,ALG,1992,2.0,1.0,0.0,1.0
2,ALG,1996,3.0,2.0,0.0,1.0
3,ALG,2000,5.0,1.0,1.0,3.0
4,ALG,2008,2.0,0.0,1.0,1.0
...,...,...,...,...,...,...
1315,NZL,2004,5.0,3.0,2.0,0.0
1316,NZL,2008,9.0,3.0,2.0,4.0
1317,NZL,2012,13.0,6.0,2.0,5.0
1318,NZL,2016,18.0,4.0,9.0,5.0


In [157]:
historical_medal_tally_relevant_case_2

,country_code,Year,Total,Gold,Silver,Bronze
0,ALG,1984,2.0,0.0,0.0,2.0
1,ALG,1992,2.0,1.0,0.0,1.0
2,ALG,1996,3.0,2.0,0.0,1.0
3,ALG,2000,5.0,1.0,1.0,3.0
4,ALG,2008,2.0,0.0,1.0,1.0
...,...,...,...,...,...,...
1322,SOL,-1,-1.0,-1.0,-1.0,-1.0
1323,SAM,2008,1.0,0.0,1.0,0.0
1324,TGA,1996,1.0,0.0,1.0,0.0
1325,TUV,-1,-1.0,-1.0,-1.0,-1.0


A quick look at the countries before we do the learning. We can see that the second test case has many more countries because we didn't filter anything out.

In [158]:
paris_2024_medal_tally_relevant = paris_2024_medal_tally_df[['country_code', 'Total', 'Gold', 'Silver', 'Bronze']]
paris_2024_medal_tally_relevant_case_2 = paris_2024_medal_tally_case_2_df[['country_code', 'Total', 'Gold', 'Silver', 'Bronze']]
paris_2024_unique_codes = paris_2024_medal_tally_relevant['country_code'].unique()
historical_medal_tally_unique_codes = historical_medal_tally_relevant['country_code'].unique()
paris_2024_unique_codes_case_2 = paris_2024_medal_tally_relevant_case_2['country_code'].unique()
historical_medal_tally_unique_codes_case_2 = historical_medal_tally_relevant_case_2['country_code'].unique()


paris_2024_unique_codes, paris_2024_unique_codes_case_2

(array(['ALG', 'EGY', 'ETH', 'KEN', 'MAR', 'NGR', 'RSA', 'TUN', 'ARG',
        'BAH', 'BRA', 'CAN', 'COL', 'CUB', 'DOM', 'JAM', 'MEX', 'TTO',
        'USA', 'VEN', 'CHN', 'KOR', 'HKG', 'IND', 'INA', 'IRI', 'JPN',
        'KAZ', 'MAS', 'MGL', 'PRK', 'TPE', 'THA', 'UZB', 'ARM', 'AUT',
        'AZE', 'BEL', 'BUL', 'CRO', 'CZE', 'DEN', 'ESP', 'EST', 'FIN',
        'FRA', 'GEO', 'GER', 'GBR', 'GRE', 'HUN', 'IRL', 'ISR', 'ITA',
        'LAT', 'LTU', 'NED', 'NOR', 'POL', 'POR', 'ROU', 'SRB', 'SVK',
        'SLO', 'SWE', 'SUI', 'TUR', 'UKR', 'AUS', 'NZL'], dtype=object),
 array(['ALG', 'ANG', 'BEN', 'BOT', 'BUR', 'BDI', 'CMR', 'CPV', 'CAF',
        'CHA', 'COM', 'CGO', 'COD', 'CIV', 'DJI', 'EGY', 'ERI', 'SWZ',
        'ETH', 'GAB', 'GAM', 'GHA', 'GUI', 'GBS', 'GEQ', 'KEN', 'LES',
        'LBR', 'LBA', 'MAD', 'MAW', 'MLI', 'MAR', 'MRI', 'MTN', 'MOZ',
        'NAM', 'NIG', 'NGR', 'UGA', 'RWA', 'STP', 'SEN', 'SEY', 'SLE',
        'SOM', 'RSA', 'SSD', 'SUD', 'TAN', 'TOG', 'TUN', 'ZAM', 'ZIM',
    

Remember, this is sequential, so just run everything one by one. The DF views are mainly for debugging. Can you find China?

In [164]:
# Preparing to merge BERT features with medal tally data
# We already have the country_names from BERT feature extraction which corresponds to the country names
# We will match these with country codes from the medal tally data

# First, let's create a DataFrame for the BERT features with corresponding country codes
bert_feature_df = pd.DataFrame.from_dict(country_features, orient='index')
num_features = len(next(iter(country_features.values())))
bert_feature_df.columns = [f'feature_{i}' for i in range(num_features)]
bert_feature_df['country_code'] = bert_feature_df.index

bert_feature_df.reset_index(drop=True, inplace=True)

# Drop rows with missing country codes (countries that are not in the medal tally data)
bert_feature_df.dropna(subset=['country_code'], inplace=True)

# Now, let's merge the BERT features with both historical and Paris 2024 medal data
# Case 1
historical_data_with_features = pd.merge(historical_medal_tally_relevant, bert_feature_df, on='country_code', how='inner')
paris_2024_data_with_features = pd.merge(paris_2024_medal_tally_relevant, bert_feature_df, on='country_code', how='inner')
# Case 2
historical_data_with_features_case_2 = pd.merge(historical_medal_tally_relevant_case_2, bert_feature_df, on='country_code', how='inner')
paris_2024_data_with_features_case_2 = pd.merge(paris_2024_medal_tally_relevant_case_2, bert_feature_df, on='country_code', how='inner')

historical_data_with_features.shape, paris_2024_data_with_features.shape, historical_data_with_features_case_2.shape, paris_2024_data_with_features_case_2.shape


((1081, 774), (70, 773), (1327, 774), (204, 773))

In [165]:
historical_data_with_features = pd.DataFrame(historical_data_with_features)
historical_data_with_features

,country_code,Year,Total,Gold,Silver,Bronze,feature_0,feature_1,feature_2,feature_3,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,ALG,1984,2.0,0.0,0.0,2.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
1,ALG,1992,2.0,1.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
2,ALG,1996,3.0,2.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
3,ALG,2000,5.0,1.0,1.0,3.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
4,ALG,2008,2.0,0.0,1.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1076,NZL,2004,5.0,3.0,2.0,0.0,-0.230367,0.046630,0.008161,-0.178960,...,0.035039,-0.017088,0.203934,-0.502458,-0.058443,-0.757425,0.182451,-0.143095,-0.097160,0.334275
1077,NZL,2008,9.0,3.0,2.0,4.0,-0.230367,0.046630,0.008161,-0.178960,...,0.035039,-0.017088,0.203934,-0.502458,-0.058443,-0.757425,0.182451,-0.143095,-0.097160,0.334275
1078,NZL,2012,13.0,6.0,2.0,5.0,-0.230367,0.046630,0.008161,-0.178960,...,0.035039,-0.017088,0.203934,-0.502458,-0.058443,-0.757425,0.182451,-0.143095,-0.097160,0.334275
1079,NZL,2016,18.0,4.0,9.0,5.0,-0.230367,0.046630,0.008161,-0.178960,...,0.035039,-0.017088,0.203934,-0.502458,-0.058443,-0.757425,0.182451,-0.143095,-0.097160,0.334275


In [166]:
historical_data_with_features_case_2 = pd.DataFrame(historical_data_with_features_case_2)
historical_data_with_features_case_2

,country_code,Year,Total,Gold,Silver,Bronze,feature_0,feature_1,feature_2,feature_3,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,ALG,1984,2.0,0.0,0.0,2.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
1,ALG,1992,2.0,1.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
2,ALG,1996,3.0,2.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
3,ALG,2000,5.0,1.0,1.0,3.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
4,ALG,2008,2.0,0.0,1.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1322,SOL,-1,-1.0,-1.0,-1.0,-1.0,-0.216952,-0.029954,0.053713,-0.061065,...,-0.068429,0.028338,0.259332,-0.282562,-0.125553,-0.452995,-0.001912,-0.099491,-0.017073,0.296681
1323,SAM,2008,1.0,0.0,1.0,0.0,-0.107072,0.095541,0.039241,-0.190647,...,-0.075343,0.154960,0.312662,-0.548999,-0.194812,-0.554261,0.070452,-0.052134,-0.047149,0.415280
1324,TGA,1996,1.0,0.0,1.0,0.0,-0.183574,-0.019720,0.021747,-0.132610,...,-0.144812,0.119026,0.315878,-0.415945,-0.104811,-0.592223,0.093978,-0.224056,-0.108927,0.425377
1325,TUV,-1,-1.0,-1.0,-1.0,-1.0,-0.128275,0.064639,0.122194,-0.044439,...,0.025434,0.020160,0.245556,-0.472722,-0.151195,-0.494881,0.044703,-0.111421,0.022991,0.363630


In [167]:
paris_2024_data_with_features = pd.DataFrame(paris_2024_data_with_features)
paris_2024_data_with_features

,country_code,Total,Gold,Silver,Bronze,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,ALG,3.0,2.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,0.137929,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
1,EGY,3.0,1.0,1.0,1.0,-0.465102,0.146367,-0.131519,-0.321259,0.009605,...,-0.100259,0.216268,0.106795,-0.342460,0.065206,-0.768027,0.079232,-0.363963,0.063853,0.235727
2,ETH,4.0,1.0,3.0,0.0,-0.409535,0.137565,-0.043246,-0.139794,0.046535,...,-0.037561,0.126538,0.145211,-0.145815,-0.052923,-0.547403,0.042519,-0.345376,0.040223,0.112593
3,KEN,11.0,4.0,2.0,5.0,-0.429986,-0.040282,-0.015930,-0.184996,0.135444,...,-0.102464,-0.126728,0.123888,-0.325692,-0.017184,-0.521751,0.115374,-0.270096,0.059250,0.215744
4,MAR,2.0,1.0,0.0,1.0,-0.481812,0.031398,-0.029591,-0.244370,0.209804,...,-0.199334,0.214556,0.214557,-0.366538,-0.042957,-0.733670,0.098782,-0.338699,-0.027667,0.296554
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,SUI,8.0,1.0,2.0,5.0,-0.526929,0.023256,0.113287,-0.192864,0.065293,...,0.011658,0.150495,0.219954,-0.333502,0.104974,-0.592063,0.154276,-0.173147,0.064206,0.257630
66,TUR,8.0,0.0,3.0,5.0,-0.390875,0.143982,-0.185502,-0.152473,0.117819,...,-0.102327,0.038741,0.163614,-0.324636,-0.084870,-0.576549,0.056370,-0.379389,0.005728,0.274972
67,UKR,12.0,3.0,5.0,4.0,-0.414747,-0.070224,0.015344,-0.154040,0.086212,...,-0.179419,0.191441,0.151555,-0.249487,-0.094757,-0.643935,0.174749,-0.401643,-0.095841,0.292428
68,AUS,53.0,18.0,19.0,16.0,-0.399308,0.016281,0.165232,-0.058390,0.004677,...,-0.096388,0.015516,0.090935,-0.336500,0.089895,-0.675746,0.157005,-0.149723,-0.009941,0.211812


In [168]:
paris_2024_data_with_features_case_2 = pd.DataFrame(paris_2024_data_with_features_case_2)
paris_2024_data_with_features_case_2

,country_code,Total,Gold,Silver,Bronze,feature_0,feature_1,feature_2,feature_3,feature_4,...,feature_758,feature_759,feature_760,feature_761,feature_762,feature_763,feature_764,feature_765,feature_766,feature_767
0,ALG,3.0,2.0,0.0,1.0,-0.481400,0.065026,-0.187783,-0.258769,0.137929,...,-0.210599,0.171756,0.168706,-0.478434,-0.021412,-0.673022,0.176665,-0.294029,0.007613,0.317239
1,ANG,-1.0,-1.0,-1.0,-1.0,-0.531567,-0.169313,-0.094242,-0.033409,0.296487,...,-0.097449,0.125326,0.197422,-0.428187,-0.046271,-0.636142,-0.062633,-0.070587,-0.054669,0.242843
2,BEN,-1.0,-1.0,-1.0,-1.0,-0.489794,0.028737,-0.049838,-0.189908,0.169756,...,-0.141605,0.104256,0.049683,-0.457860,0.043841,-0.658490,0.022212,-0.060198,0.025601,0.235664
3,BOT,2.0,1.0,1.0,0.0,-0.466590,-0.096590,0.113659,-0.024595,0.127837,...,-0.101390,0.209467,0.119671,-0.498387,0.091250,-0.544316,-0.032646,-0.149083,0.001064,0.215812
4,BUR,-1.0,-1.0,-1.0,-1.0,-0.431025,-0.038477,-0.113337,-0.156375,0.116831,...,-0.242285,0.227782,0.051940,-0.384393,0.074634,-0.731360,-0.084426,-0.226577,-0.174809,0.141097
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,SOL,-1.0,-1.0,-1.0,-1.0,-0.216952,-0.029954,0.053713,-0.061065,0.143249,...,-0.068429,0.028338,0.259332,-0.282562,-0.125553,-0.452995,-0.001912,-0.099491,-0.017073,0.296681
200,SAM,-1.0,-1.0,-1.0,-1.0,-0.107072,0.095541,0.039241,-0.190647,0.231200,...,-0.075343,0.154960,0.312662,-0.548999,-0.194812,-0.554261,0.070452,-0.052134,-0.047149,0.415280
201,TGA,-1.0,-1.0,-1.0,-1.0,-0.183574,-0.019720,0.021747,-0.132610,0.101105,...,-0.144812,0.119026,0.315878,-0.415945,-0.104811,-0.592223,0.093978,-0.224056,-0.108927,0.425377
202,TUV,-1.0,-1.0,-1.0,-1.0,-0.128275,0.064639,0.122194,-0.044439,0.235125,...,0.025434,0.020160,0.245556,-0.472722,-0.151195,-0.494881,0.044703,-0.111421,0.022991,0.363630


#THE MAIN EVENT

Ok, we made it so far.
Let's scale the data and it into train and validation sets. We'll use the ACTUAL Paris results as a test set(should be ready before our presentation). How cool is that?

In [170]:
scaler1 = StandardScaler()
scaler2 = StandardScaler()
# Splitting features and target (Total medals)
X_train = historical_data_with_features.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
X_train_2 = historical_data_with_features_case_2.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])

X_train = X_train.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN
X_train_2 = X_train_2.apply(pd.to_numeric, errors='coerce')  # Convert non-numeric values to NaN

X_train_scaled = scaler1.fit_transform(X_train)
X_train_tensor_scaled = torch.tensor(X_train_scaled, dtype=torch.float32).to(device)

X_train_scaled_2 = scaler2.fit_transform(X_train_2)
X_train_tensor_scaled_2 = torch.tensor(X_train_scaled_2, dtype=torch.float32).to(device)

y_train = historical_data_with_features[['Total', 'Gold', 'Silver', 'Bronze']]
y_train_2 = historical_data_with_features_case_2[['Total', 'Gold', 'Silver', 'Bronze']]

y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).to(device)
y_train_tensor_2 = torch.tensor(y_train_2.values, dtype=torch.float32).to(device)

paris_2024_data_with_features_sorted = paris_2024_data_with_features.sort_values(by='Gold', ascending=False)
paris_2024_data_with_features_sorted_2 = paris_2024_data_with_features_case_2.sort_values(by='Gold', ascending=False)

X_test = paris_2024_data_with_features_sorted.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
X_test_2 = paris_2024_data_with_features_sorted_2.drop(columns=['Total', 'Gold', 'Silver', 'Bronze', 'country_code'])
test_countries = paris_2024_data_with_features_sorted['country_code']
test_countries_2 = paris_2024_data_with_features_sorted_2['country_code']

# Add a constant 'Year' column to X_test with value 2024
X_test.insert(0, 'Year', 2024)
X_test_2.insert(0, 'Year', 2024)
#Scale and transform
X_test_scaled = scaler1.transform(X_test)  # Use the same scaler fitted on training data
X_test_tensor_scaled = torch.tensor(X_test_scaled, dtype=torch.float32).to(device)

X_test_scaled_2 = scaler2.transform(X_test_2)  # Use the same scaler fitted on training data
X_test_tensor_scaled_2 = torch.tensor(X_test_scaled_2, dtype=torch.float32).to(device)

y_test = paris_2024_data_with_features_sorted[['Total', 'Gold', 'Silver', 'Bronze']]
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).to(device)

y_test_2 = paris_2024_data_with_features_sorted_2[['Total', 'Gold', 'Silver', 'Bronze']]
y_test_tensor_2 = torch.tensor(y_test_2.values, dtype=torch.float32).to(device)






#TRAINING

We'll use Optuna in order to tuna Hyperparameters and find the best combo. Then we'll train our network and see the results. It's as simple as that.

In [171]:
import optuna

# Splitting train and validation data
X_train_sub, X_val, y_train_sub, y_val = train_test_split(X_train_tensor_scaled, y_train_tensor, test_size=0.2, random_state=42)

def objective(trial):
    # Define the hyperparameter search space
    n_units_1 = trial.suggest_int('n_units_1', 256, 1024)
    n_units_2 = trial.suggest_int('n_units_2', 128, 512)
    n_units_3 = trial.suggest_int('n_units_3', 64, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    # Building the model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(X_train_sub.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(X_train_sub).to(device)
        loss = criterion(outputs.squeeze(), y_train_sub)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        loss.backward()
        optimizer.step()

    # Evaluate the model on validation data
    with torch.no_grad():
        val_outputs = model(X_val).to(device)
        val_outputs = torch.round(val_outputs)  # Round the predictions to the nearest integer
        val_mae = {}
        val_accuracy = {}
        tolerance = 0.2
        tolerance_biases = [7.0, 3.0, 3.0, 3.0]
        for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
          mae = torch.mean(torch.abs(val_outputs[:,i] - y_val[:,i])).item()
          val_mae[medal_type] = mae
          correct = torch.sum(torch.abs(val_outputs[:,i] - y_val[:,i]) <= torch.tensor(tolerance_biases[i]).to(device)*y_val[:,i])
          accuracy = correct.item() / len(y_val) * 100
          val_accuracy[medal_type] = accuracy
          val_loss = criterion(val_outputs[:,i], y_val[:,i])
        print(f"Validation Loss: {val_loss} ; Validation Accuracy: {accuracy}%")

    return sum(val_mae.values())  # Return the validation MAE as a Python numbe

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Build and evaluate the model with the best hyperparameters
n_units_1 = best_params['n_units_1']
n_units_2 = best_params['n_units_2']
n_units_3 = best_params['n_units_3']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']

    # Building the model with trial hyperparameters
best_model = nn.Sequential(
        nn.Linear(X_train_tensor_scaled.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

# Train with the entire training data and evaluate on the test set
for epoch in range(50):
    optimizer.zero_grad()
    outputs = best_model(X_train_tensor_scaled).to(device)
    loss = criterion(outputs.squeeze(), y_train_tensor)
    loss.backward()
    optimizer.step()

    # Evaluate the model on test data
with torch.no_grad():
    test_mae = {}
    test_accuracy = {}
    test_outputs = best_model(X_test_tensor_scaled).to(device)
    test_outputs = torch.round(test_outputs)  # Round the predictions to the nearest integer
    tolerance_biases = [7.0, 3.0, 3.0, 3.0]
    for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
      test_loss = criterion(test_outputs[:,i], y_test_tensor[:,i])
      tolerance = 0.25
      correct = torch.sum(torch.abs(test_outputs[:,i] - y_test_tensor[:,i]) <= torch.max(tolerance*y_test_tensor[:,i], torch.tensor(tolerance_biases[i]).to(device)))
      accuracy = correct.item() / len(y_test) * 100
      test_accuracy[medal_type] = accuracy
      mae = torch.mean(torch.abs(test_outputs[:,i] - y_test_tensor[:,i])).item()  # Calculate MAE
      test_mae[medal_type] = mae


print(f"Test MAE: {test_mae}")
print(f"Test Loss: {test_loss} ; Test Accuracy: {test_accuracy}%")


[I 2024-08-18 14:11:25,594] A new study created in memory with name: no-name-a045d54a-b3b0-4bb2-9a1c-d746e19859c7
[I 2024-08-18 14:11:25,702] Trial 0 finished with value: 17.52534532546997 and parameters: {'n_units_1': 1024, 'n_units_2': 154, 'n_units_3': 96, 'dropout_rate': 0.31363767518388097, 'learning_rate': 0.000356703928938053}. Best is trial 0 with value: 17.52534532546997.
[I 2024-08-18 14:11:25,795] Trial 1 finished with value: 21.184332370758057 and parameters: {'n_units_1': 670, 'n_units_2': 145, 'n_units_3': 90, 'dropout_rate': 0.290451126189584, 'learning_rate': 0.0006197938672130309}. Best is trial 0 with value: 17.52534532546997.


Epoch 1, Loss: 212.74293518066406
Epoch 2, Loss: 211.5515899658203
Epoch 3, Loss: 210.41656494140625
Epoch 4, Loss: 209.0385284423828
Epoch 5, Loss: 207.35662841796875
Epoch 6, Loss: 205.5083770751953
Epoch 7, Loss: 203.1262969970703
Epoch 8, Loss: 200.60855102539062
Epoch 9, Loss: 197.6058349609375
Epoch 10, Loss: 193.79930114746094
Epoch 11, Loss: 189.73095703125
Epoch 12, Loss: 185.3426971435547
Epoch 13, Loss: 179.6175994873047
Epoch 14, Loss: 173.4788360595703
Epoch 15, Loss: 167.0694122314453
Epoch 16, Loss: 159.98927307128906
Epoch 17, Loss: 152.24635314941406
Epoch 18, Loss: 143.54420471191406
Epoch 19, Loss: 135.39930725097656
Epoch 20, Loss: 124.47371673583984
Validation Loss: 20.80645179748535 ; Validation Accuracy: 79.72350230414746%
Epoch 1, Loss: 212.831787109375
Epoch 2, Loss: 211.3584747314453
Epoch 3, Loss: 209.78379821777344
Epoch 4, Loss: 207.9241180419922
Epoch 5, Loss: 205.386474609375
Epoch 6, Loss: 202.551513671875
Epoch 7, Loss: 198.55148315429688
Epoch 8, Loss:

[I 2024-08-18 14:11:25,917] Trial 2 finished with value: 20.165898323059082 and parameters: {'n_units_1': 859, 'n_units_2': 330, 'n_units_3': 132, 'dropout_rate': 0.45539826332309385, 'learning_rate': 0.00043399830091555577}. Best is trial 0 with value: 17.52534532546997.
[I 2024-08-18 14:11:25,995] Trial 3 finished with value: 15.552995681762695 and parameters: {'n_units_1': 260, 'n_units_2': 464, 'n_units_3': 120, 'dropout_rate': 0.3134118086960192, 'learning_rate': 0.0007916742584223209}. Best is trial 3 with value: 15.552995681762695.


Epoch 1, Loss: 212.49464416503906
Epoch 2, Loss: 210.7615966796875
Epoch 3, Loss: 208.64370727539062
Epoch 4, Loss: 206.13973999023438
Epoch 5, Loss: 202.53887939453125
Epoch 6, Loss: 198.3521728515625
Epoch 7, Loss: 193.32557678222656
Epoch 8, Loss: 186.17115783691406
Epoch 9, Loss: 178.5065460205078
Epoch 10, Loss: 169.0978546142578
Epoch 11, Loss: 158.4095458984375
Epoch 12, Loss: 146.06109619140625
Epoch 13, Loss: 133.8997802734375
Epoch 14, Loss: 120.94416809082031
Epoch 15, Loss: 110.91838836669922
Epoch 16, Loss: 102.48157501220703
Epoch 17, Loss: 98.06615447998047
Epoch 18, Loss: 91.72135162353516
Epoch 19, Loss: 89.33110809326172
Epoch 20, Loss: 85.55243682861328
Validation Loss: 23.161291122436523 ; Validation Accuracy: 72.81105990783409%
Epoch 1, Loss: 212.8304443359375
Epoch 2, Loss: 211.1222686767578
Epoch 3, Loss: 209.0164031982422
Epoch 4, Loss: 206.12779235839844
Epoch 5, Loss: 202.03750610351562
Epoch 6, Loss: 196.3264617919922
Epoch 7, Loss: 189.36045837402344
Epoch 8

[I 2024-08-18 14:11:26,079] Trial 4 finished with value: 18.470045804977417 and parameters: {'n_units_1': 266, 'n_units_2': 350, 'n_units_3': 236, 'dropout_rate': 0.34925086395673954, 'learning_rate': 0.00027374801554431056}. Best is trial 3 with value: 15.552995681762695.
[I 2024-08-18 14:11:26,189] Trial 5 finished with value: 18.82488441467285 and parameters: {'n_units_1': 715, 'n_units_2': 280, 'n_units_3': 193, 'dropout_rate': 0.2937665311594404, 'learning_rate': 0.0002093529223397296}. Best is trial 3 with value: 15.552995681762695.


...

Epoch 7, Loss: 169.35446166992188
Epoch 8, Loss: 153.0353240966797
Epoch 9, Loss: 134.2012176513672
Epoch 10, Loss: 115.50457763671875
Epoch 11, Loss: 100.2251968383789
Epoch 12, Loss: 88.90670013427734
Epoch 13, Loss: 88.96238708496094
Epoch 14, Loss: 85.37958526611328
Epoch 15, Loss: 81.547119140625
Epoch 16, Loss: 74.68029022216797
Epoch 17, Loss: 68.10894012451172
Epoch 18, Loss: 62.708770751953125
Epoch 19, Loss: 61.3343391418457
Epoch 20, Loss: 61.830596923828125
Validation Loss: 15.193548202514648 ; Validation Accuracy: 81.5668202764977%
Epoch 1, Loss: 212.3166961669922
Epoch 2, Loss: 209.57101440429688
Epoch 3, Loss: 205.60214233398438
Epoch 4, Loss: 199.8649139404297
Epoch 5, Loss: 191.2379150390625
Epoch 6, Loss: 179.7279510498047
Epoch 7, Loss: 165.10191345214844
Epoch 8, Loss: 147.38357543945312
Epoch 9, Loss: 127.49555206298828
Epoch 10, Loss: 108.59539794921875
Epoch 11, Loss: 98.74280548095703
Epoch 12, Loss: 96.89393615722656
Epoch 13, Loss: 90.89569091796875
Epoch 14, 

[I 2024-08-18 14:11:30,112] Trial 46 finished with value: 17.96774196624756 and parameters: {'n_units_1': 268, 'n_units_2': 465, 'n_units_3': 75, 'dropout_rate': 0.2522982894335175, 'learning_rate': 0.000399122925374557}. Best is trial 19 with value: 13.95852541923523.
[I 2024-08-18 14:11:30,230] Trial 47 finished with value: 14.423963069915771 and parameters: {'n_units_1': 870, 'n_units_2': 438, 'n_units_3': 86, 'dropout_rate': 0.2990249153009773, 'learning_rate': 0.0009358447116853913}. Best is trial 19 with value: 13.95852541923523.


Epoch 12, Loss: 199.7741241455078
Epoch 13, Loss: 197.1249237060547
Epoch 14, Loss: 193.96115112304688
Epoch 15, Loss: 189.72994995117188
Epoch 16, Loss: 185.41921997070312
Epoch 17, Loss: 180.1576690673828
Epoch 18, Loss: 174.61964416503906
Epoch 19, Loss: 167.62710571289062
Epoch 20, Loss: 161.41265869140625
Validation Loss: 27.44239616394043 ; Validation Accuracy: 83.41013824884793%
Epoch 1, Loss: 213.1959686279297
Epoch 2, Loss: 209.2243194580078
Epoch 3, Loss: 203.6632537841797
Epoch 4, Loss: 194.86940002441406
Epoch 5, Loss: 182.1730499267578
Epoch 6, Loss: 164.5141143798828
Epoch 7, Loss: 143.46266174316406
Epoch 8, Loss: 122.51454162597656
Epoch 9, Loss: 105.34613800048828
Epoch 10, Loss: 97.87371063232422
Epoch 11, Loss: 90.57041931152344
Epoch 12, Loss: 82.23995208740234
Epoch 13, Loss: 73.57503509521484
Epoch 14, Loss: 67.61985778808594
Epoch 15, Loss: 61.54829406738281
Epoch 16, Loss: 59.84855270385742
Epoch 17, Loss: 61.65462112426758
Epoch 18, Loss: 64.342529296875
Epoch 

[I 2024-08-18 14:11:30,342] Trial 48 finished with value: 14.281105995178223 and parameters: {'n_units_1': 790, 'n_units_2': 496, 'n_units_3': 119, 'dropout_rate': 0.47545115541130756, 'learning_rate': 0.0007966718872323274}. Best is trial 19 with value: 13.95852541923523.
[I 2024-08-18 14:11:30,431] Trial 49 finished with value: 14.0506911277771 and parameters: {'n_units_1': 404, 'n_units_2': 410, 'n_units_3': 100, 'dropout_rate': 0.2138963707028985, 'learning_rate': 0.0008778996500271026}. Best is trial 19 with value: 13.95852541923523.


Epoch 6, Loss: 162.1790008544922
Epoch 7, Loss: 141.11546325683594
Epoch 8, Loss: 118.0960922241211
Epoch 9, Loss: 102.34577941894531
Epoch 10, Loss: 94.1323013305664
Epoch 11, Loss: 91.29647827148438
Epoch 12, Loss: 91.6722183227539
Epoch 13, Loss: 80.49636840820312
Epoch 14, Loss: 75.15314483642578
Epoch 15, Loss: 63.76881790161133
Epoch 16, Loss: 62.77628707885742
Epoch 17, Loss: 62.520408630371094
Epoch 18, Loss: 61.8804817199707
Epoch 19, Loss: 61.99791717529297
Epoch 20, Loss: 62.967918395996094
Validation Loss: 18.594470977783203 ; Validation Accuracy: 86.17511520737328%
Epoch 1, Loss: 211.54888916015625
Epoch 2, Loss: 209.1127471923828
Epoch 3, Loss: 205.9221649169922
Epoch 4, Loss: 201.35009765625
Epoch 5, Loss: 194.5211944580078
Epoch 6, Loss: 184.9001007080078
Epoch 7, Loss: 172.1671600341797
Epoch 8, Loss: 156.40187072753906
Epoch 9, Loss: 137.91390991210938
Epoch 10, Loss: 117.22349548339844
Epoch 11, Loss: 101.85609436035156
Epoch 12, Loss: 92.280029296875
Epoch 13, Loss:

Let's see what the results are. We already have predictions for each medal separately and total medals in the previous cell. Pretty accurate, right? Wait till you see when happens in the second test case.
P.S. if we take the separate medals and sum them up, we'll do a bit better:

In [182]:
medal_manual_sum = torch.sum(test_outputs[:,1:], dim=1)
sum_correct = torch.sum(torch.abs(medal_manual_sum - y_test_tensor[:,0]) <= torch.max(tolerance*y_test_tensor[:,0], torch.tensor(7.0).to(device)))
sum_accuracy = sum_correct.item() / len(y_test) * 100

print(f"Test Accuracy for Manual Medal Total: {sum_accuracy}%")

Test Accuracy for Manual Medal Total: 77.14285714285715%


No let's see the predicted table!

In [181]:
test_countries = np.array(test_countries)  # Convert to a numpy array if it's not already

# Create a DataFrame from the test predictions
predictions_df = pd.DataFrame({
    'Country': test_countries,
    'Predicted Total': test_outputs[:, 0].cpu().numpy(),
    'Predicted Gold': test_outputs[:, 1].cpu().numpy(),
    'Predicted Silver': test_outputs[:, 2].cpu().numpy(),
    'Predicted Bronze': test_outputs[:, 3].cpu().numpy(),
    'Total Predicted': medal_manual_sum.cpu().numpy()
})

predictions_df

,Country,Predicted Total,Predicted Gold,Predicted Silver,Predicted Bronze,Total Predicted
0,USA,91.0,38.0,29.0,30.0,97.0
1,CHN,65.0,24.0,21.0,21.0,66.0
2,JPN,20.0,7.0,7.0,7.0,21.0
3,AUS,21.0,6.0,7.0,7.0,20.0
4,FRA,30.0,10.0,10.0,11.0,31.0
...,...,...,...,...,...,...
65,VEN,2.0,0.0,0.0,1.0,1.0
66,FIN,12.0,3.0,4.0,5.0,12.0
67,EST,4.0,1.0,1.0,1.0,3.0
68,LAT,3.0,1.0,1.0,1.0,3.0


And for reference, the actual table(added at the end of the Olympics):

In [174]:
actual_df = pd.DataFrame({
    'Country': test_countries,
    'Actual Total': y_test['Total'].values,
    'Actual Gold': y_test['Gold'].values,
    'Actual Silver': y_test['Silver'].values,
    'Actual Bronze': y_test['Bronze'].values
})

actual_df

,Country,Actual Total,Actual Gold,Actual Silver,Actual Bronze
0,USA,126.0,40.0,44.0,42.0
1,CHN,91.0,40.0,27.0,24.0
2,JPN,45.0,20.0,12.0,13.0
3,AUS,53.0,18.0,19.0,16.0
4,FRA,64.0,16.0,26.0,22.0
...,...,...,...,...,...
65,VEN,-1.0,-1.0,-1.0,-1.0
66,FIN,-1.0,-1.0,-1.0,-1.0
67,EST,-1.0,-1.0,-1.0,-1.0
68,LAT,-1.0,-1.0,-1.0,-1.0


Ok, now for the second test case. Here we didn't filter anything out, just looked at all the participating countries in Paris.

In [175]:
# Test case 2:

# Splitting train and validation data
X_train_sub_2, X_val_2, y_train_sub_2, y_val_2 = train_test_split(X_train_tensor_scaled_2, y_train_tensor_2, test_size=0.2, random_state=42)

def objective(trial):
    # Define the hyperparameter search space
    n_units_1 = trial.suggest_int('n_units_1', 256, 1024)
    n_units_2 = trial.suggest_int('n_units_2', 128, 512)
    n_units_3 = trial.suggest_int('n_units_3', 64, 256)
    dropout_rate = trial.suggest_float('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-3)

    # Building the model with trial hyperparameters
    model = nn.Sequential(
        nn.Linear(X_train_sub.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # Train the model
    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(X_train_sub_2).to(device)
        loss = criterion(outputs.squeeze(), y_train_sub_2)
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
        loss.backward()
        optimizer.step()

    # Evaluate the model on validation data
    with torch.no_grad():
        val_outputs = model(X_val_2).to(device)
        val_outputs = torch.round(val_outputs)  # Round the predictions to the nearest integer
        val_mae = {}
        val_accuracy = {}
        tolerance = 0.2
        tolerance_biases = [7.0, 3.0, 3.0, 3.0]
        for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
          mae = torch.mean(torch.abs(val_outputs[:,i] - y_val_2[:,i])).item()
          val_mae[medal_type] = mae
          correct = torch.sum(torch.abs(val_outputs[:,i] - y_val_2[:,i]) <= torch.tensor(tolerance_biases[i]).to(device)*y_val_2[:,i])
          accuracy = correct.item() / len(y_val_2) * 100
          val_accuracy[medal_type] = accuracy
          val_loss = criterion(val_outputs[:,i], y_val_2[:,i])
        print(f"Validation Loss: {val_loss} ; Validation Accuracy: {accuracy}%")

    return sum(val_mae.values())  # Return the validation MAE as a Python numbe

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

# Get the best hyperparameters
best_params = study.best_params
print("Best hyperparameters: ", best_params)

# Build and evaluate the model with the best hyperparameters
n_units_1 = best_params['n_units_1']
n_units_2 = best_params['n_units_2']
n_units_3 = best_params['n_units_3']
dropout_rate = best_params['dropout_rate']
learning_rate = best_params['learning_rate']

    # Building the model with trial hyperparameters
best_model = nn.Sequential(
        nn.Linear(X_train_tensor_scaled_2.shape[1], n_units_1),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_1, n_units_2),
        nn.ReLU(),
        nn.Dropout(dropout_rate),
        nn.Linear(n_units_2, n_units_3),
        nn.ReLU(),
        nn.Linear(n_units_3, 4)  # Output layer for regression
    ).to(device)  # Move the model to the GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(best_model.parameters(), lr=learning_rate)

# Train with the entire training data and evaluate on the test set
for epoch in range(50):
    optimizer.zero_grad()
    outputs = best_model(X_train_tensor_scaled_2).to(device)
    loss = criterion(outputs.squeeze(), y_train_tensor_2)
    loss.backward()
    optimizer.step()

    # Evaluate the model on test data
with torch.no_grad():
    test_mae = {}
    test_accuracy = {}
    test_outputs_2 = best_model(X_test_tensor_scaled_2).to(device)
    test_outputs_2 = torch.round(test_outputs_2)  # Round the predictions to the nearest integer
    tolerance_biases = [7.0, 3.0, 3.0, 3.0]
    for i, medal_type in enumerate(['Total', 'Gold', 'Silver', 'Bronze']):
      test_loss = criterion(test_outputs_2[:,i], y_test_tensor_2[:,i])
      tolerance = 0.25
      correct = torch.sum(torch.abs(test_outputs_2[:,i] - y_test_tensor_2[:,i]) <= torch.max(tolerance*y_test_tensor_2[:,i], torch.tensor(tolerance_biases[i]).to(device)))
      accuracy = correct.item() / len(y_test_2) * 100
      test_accuracy[medal_type] = accuracy
      mae = torch.mean(torch.abs(test_outputs_2[:,i] - y_test_tensor_2[:,i])).item()  # Calculate MAE
      test_mae[medal_type] = mae


print(f"Test MAE: {test_mae}")
print(f"Test Loss: {test_loss} ; Test Accuracy: {test_accuracy}%")


[I 2024-08-18 14:12:12,477] A new study created in memory with name: no-name-c13fad0a-d6c5-46c7-b4a4-d52b954762ed
[I 2024-08-18 14:12:12,580] Trial 0 finished with value: 16.421052932739258 and parameters: {'n_units_1': 1015, 'n_units_2': 140, 'n_units_3': 101, 'dropout_rate': 0.36670868475270235, 'learning_rate': 0.0006684051592370293}. Best is trial 0 with value: 16.421052932739258.
[I 2024-08-18 14:12:12,663] Trial 1 finished with value: 15.736842393875122 and parameters: {'n_units_1': 317, 'n_units_2': 418, 'n_units_3': 229, 'dropout_rate': 0.4962210913959211, 'learning_rate': 0.0006479387719418275}. Best is trial 1 with value: 15.736842393875122.


Epoch 1, Loss: 147.78903198242188
Epoch 2, Loss: 146.39125061035156
Epoch 3, Loss: 144.77560424804688
Epoch 4, Loss: 142.39364624023438
Epoch 5, Loss: 139.35635375976562
Epoch 6, Loss: 135.76986694335938
Epoch 7, Loss: 130.5080108642578
Epoch 8, Loss: 124.65779876708984
Epoch 9, Loss: 116.93080139160156
Epoch 10, Loss: 109.33895874023438
Epoch 11, Loss: 99.15070343017578
Epoch 12, Loss: 89.80205535888672
Epoch 13, Loss: 81.64229583740234
Epoch 14, Loss: 71.92452239990234
Epoch 15, Loss: 65.83824157714844
Epoch 16, Loss: 62.65933609008789
Epoch 17, Loss: 62.86573791503906
Epoch 18, Loss: 59.81515884399414
Epoch 19, Loss: 59.43532180786133
Epoch 20, Loss: 53.21694564819336
Validation Loss: 25.169174194335938 ; Validation Accuracy: 69.54887218045113%
Epoch 1, Loss: 148.0503692626953
Epoch 2, Loss: 146.51731872558594
Epoch 3, Loss: 144.69802856445312
Epoch 4, Loss: 142.26890563964844
Epoch 5, Loss: 139.1923828125
Epoch 6, Loss: 134.78089904785156
Epoch 7, Loss: 128.78787231445312
Epoch 8, 

[I 2024-08-18 14:12:12,767] Trial 2 finished with value: 14.838346242904663 and parameters: {'n_units_1': 687, 'n_units_2': 442, 'n_units_3': 174, 'dropout_rate': 0.23276395158032703, 'learning_rate': 0.0007289771309258667}. Best is trial 2 with value: 14.838346242904663.
[I 2024-08-18 14:12:12,855] Trial 3 finished with value: 18.552631855010986 and parameters: {'n_units_1': 807, 'n_units_2': 382, 'n_units_3': 105, 'dropout_rate': 0.468257031217563, 'learning_rate': 0.0002948451024404624}. Best is trial 2 with value: 14.838346242904663.


Epoch 5, Loss: 129.93296813964844
Epoch 6, Loss: 120.19876861572266
Epoch 7, Loss: 107.9389419555664
Epoch 8, Loss: 93.4400634765625
Epoch 9, Loss: 79.29341125488281
Epoch 10, Loss: 70.45770263671875
Epoch 11, Loss: 66.07178497314453
Epoch 12, Loss: 64.57382202148438
Epoch 13, Loss: 61.35271453857422
Epoch 14, Loss: 55.069480895996094
Epoch 15, Loss: 49.28738784790039
Epoch 16, Loss: 45.09371566772461
Epoch 17, Loss: 42.62403869628906
Epoch 18, Loss: 42.96051788330078
Epoch 19, Loss: 42.59315872192383
Epoch 20, Loss: 43.59547424316406
Validation Loss: 35.962406158447266 ; Validation Accuracy: 83.83458646616542%
Epoch 1, Loss: 147.12730407714844
Epoch 2, Loss: 146.41366577148438
Epoch 3, Loss: 145.4665985107422
Epoch 4, Loss: 144.45498657226562
Epoch 5, Loss: 143.0809326171875
Epoch 6, Loss: 141.7294921875
Epoch 7, Loss: 139.9495849609375
Epoch 8, Loss: 137.9005889892578
Epoch 9, Loss: 135.71836853027344
Epoch 10, Loss: 132.6471405029297
Epoch 11, Loss: 129.40948486328125
Epoch 12, Loss

[I 2024-08-18 14:12:12,951] Trial 4 finished with value: 16.00000023841858 and parameters: {'n_units_1': 655, 'n_units_2': 428, 'n_units_3': 152, 'dropout_rate': 0.36858282066503445, 'learning_rate': 0.0005540531514784397}. Best is trial 2 with value: 14.838346242904663.
[I 2024-08-18 14:12:13,026] Trial 5 finished with value: 24.36842179298401 and parameters: {'n_units_1': 560, 'n_units_2': 201, 'n_units_3': 178, 'dropout_rate': 0.44656311304742746, 'learning_rate': 6.89951708234466e-05}. Best is trial 2 with value: 14.838346242904663.


...

Epoch 15, Loss: 45.09840393066406
Epoch 16, Loss: 44.35466384887695
Epoch 17, Loss: 44.611812591552734
Epoch 18, Loss: 43.60784912109375
Epoch 19, Loss: 44.47015380859375
Epoch 20, Loss: 41.98258590698242
Validation Loss: 28.699249267578125 ; Validation Accuracy: 82.33082706766918%
Epoch 1, Loss: 147.97027587890625
Epoch 2, Loss: 146.10096740722656
Epoch 3, Loss: 143.8246307373047
Epoch 4, Loss: 140.3936309814453
Epoch 5, Loss: 135.39614868164062
Epoch 6, Loss: 128.43687438964844
Epoch 7, Loss: 119.142822265625
Epoch 8, Loss: 107.5685806274414
Epoch 9, Loss: 95.24125671386719
Epoch 10, Loss: 82.04314422607422
Epoch 11, Loss: 73.01713562011719
Epoch 12, Loss: 68.30540466308594
Epoch 13, Loss: 65.17474365234375
Epoch 14, Loss: 62.564002990722656
Epoch 15, Loss: 57.14765930175781
Epoch 16, Loss: 52.100154876708984
Epoch 17, Loss: 47.37710952758789
Epoch 18, Loss: 44.305809020996094
Epoch 19, Loss: 44.25358581542969
Epoch 20, Loss: 43.563114166259766
Validation Loss: 33.09022521972656 ; Va

[I 2024-08-18 14:12:17,086] Trial 46 finished with value: 13.992481708526611 and parameters: {'n_units_1': 650, 'n_units_2': 496, 'n_units_3': 190, 'dropout_rate': 0.12731359438677237, 'learning_rate': 0.0009475235349649983}. Best is trial 23 with value: 13.845865249633789.
[I 2024-08-18 14:12:17,184] Trial 47 finished with value: 14.89097785949707 and parameters: {'n_units_1': 525, 'n_units_2': 337, 'n_units_3': 210, 'dropout_rate': 0.4256116224261678, 'learning_rate': 0.0007503017970822055}. Best is trial 23 with value: 13.845865249633789.


Epoch 18, Loss: 40.84159851074219
Epoch 19, Loss: 39.783077239990234
Epoch 20, Loss: 38.6664924621582
Validation Loss: 26.330827713012695 ; Validation Accuracy: 81.57894736842105%
Epoch 1, Loss: 148.5465850830078
Epoch 2, Loss: 146.63880920410156
Epoch 3, Loss: 144.54747009277344
Epoch 4, Loss: 141.28372192382812
Epoch 5, Loss: 137.03524780273438
Epoch 6, Loss: 130.82058715820312
Epoch 7, Loss: 122.3909912109375
Epoch 8, Loss: 111.75435638427734
Epoch 9, Loss: 100.4947738647461
Epoch 10, Loss: 86.3582763671875
Epoch 11, Loss: 74.31959533691406
Epoch 12, Loss: 68.49998474121094
Epoch 13, Loss: 65.14684295654297
Epoch 14, Loss: 63.55904006958008
Epoch 15, Loss: 59.59033966064453
Epoch 16, Loss: 55.96901321411133
Epoch 17, Loss: 51.7414665222168
Epoch 18, Loss: 48.082733154296875
Epoch 19, Loss: 45.26488494873047
Epoch 20, Loss: 43.849212646484375
Validation Loss: 28.060150146484375 ; Validation Accuracy: 77.81954887218046%
Epoch 1, Loss: 147.76905822753906
Epoch 2, Loss: 145.504028320312

[I 2024-08-18 14:12:17,285] Trial 48 finished with value: 14.665414094924927 and parameters: {'n_units_1': 716, 'n_units_2': 478, 'n_units_3': 229, 'dropout_rate': 0.2765806665825521, 'learning_rate': 0.0006338931012202939}. Best is trial 23 with value: 13.845865249633789.
[I 2024-08-18 14:12:17,382] Trial 49 finished with value: 14.778195858001709 and parameters: {'n_units_1': 890, 'n_units_2': 227, 'n_units_3': 200, 'dropout_rate': 0.16112352979243888, 'learning_rate': 0.0008508074629990699}. Best is trial 23 with value: 13.845865249633789.


Epoch 19, Loss: 44.405921936035156
Epoch 20, Loss: 43.14606857299805
Validation Loss: 33.582706451416016 ; Validation Accuracy: 80.82706766917293%
Epoch 1, Loss: 148.00987243652344
Epoch 2, Loss: 145.9523468017578
Epoch 3, Loss: 143.05621337890625
Epoch 4, Loss: 138.46690368652344
Epoch 5, Loss: 131.88783264160156
Epoch 6, Loss: 122.79413604736328
Epoch 7, Loss: 110.95260620117188
Epoch 8, Loss: 97.650390625
Epoch 9, Loss: 85.18917846679688
Epoch 10, Loss: 76.16691589355469
Epoch 11, Loss: 70.82804107666016
Epoch 12, Loss: 65.870361328125
Epoch 13, Loss: 60.05363845825195
Epoch 14, Loss: 55.115089416503906
Epoch 15, Loss: 49.256690979003906
Epoch 16, Loss: 46.62143325805664
Epoch 17, Loss: 44.803955078125
Epoch 18, Loss: 44.13697052001953
Epoch 19, Loss: 44.5294303894043
Epoch 20, Loss: 43.32481384277344
Validation Loss: 36.97744369506836 ; Validation Accuracy: 84.21052631578947%
Best hyperparameters:  {'n_units_1': 886, 'n_units_2': 473, 'n_units_3': 218, 'dropout_rate': 0.20568777731

Wow, that's crazy! Let's sum them up:

In [178]:
medal_manual_sum_2 = torch.sum(test_outputs_2[:,1:], dim=1)
sum_correct = torch.sum(torch.abs(medal_manual_sum_2 - y_test_tensor_2[:,0]) <= torch.max(tolerance*y_test_tensor_2[:,0], torch.tensor(7.0).to(device)))
sum_accuracy = sum_correct.item() / len(y_test_2) * 100

print(f"Test Accuracy for Manual Medal Total in Test Case 2: {sum_accuracy}%")

Test Accuracy for Manual Medal Total in Test Case 2: 91.66666666666666%


And for the medal table...

In [180]:
test_countries_2 = np.array(test_countries_2)  # Convert to a numpy array if it's not already

# Create a DataFrame from the test predictions
predictions_case_2_df = pd.DataFrame({
    'Country': test_countries_2,
    'Predicted Total': test_outputs_2[:, 0].cpu().numpy(),
    'Predicted Gold': test_outputs_2[:, 1].cpu().numpy(),
    'Predicted Silver': test_outputs_2[:, 2].cpu().numpy(),
    'Predicted Bronze': test_outputs_2[:, 3].cpu().numpy(),
    'Total Predicted': medal_manual_sum_2.cpu().numpy()
})

predictions_case_2_df

,Country,Predicted Total,Predicted Gold,Predicted Silver,Predicted Bronze,Total Predicted
0,CHN,68.0,26.0,21.0,22.0,69.0
1,USA,97.0,42.0,29.0,28.0,99.0
2,JPN,22.0,9.0,7.0,8.0,24.0
3,AUS,21.0,8.0,7.0,8.0,23.0
4,FRA,27.0,9.0,10.0,10.0,29.0
...,...,...,...,...,...,...
199,CHA,1.0,0.0,0.0,0.0,0.0
200,COM,0.0,0.0,0.0,0.0,0.0
201,GUM,0.0,0.0,0.0,0.0,0.0
202,SAM,1.0,0.0,0.0,0.0,0.0


Compare to the actual results:

In [111]:
actual_df_2 = pd.DataFrame({
    'Country': test_countries_2,
    'Actual Total': y_test_2['Total'].values,
    'Actual Gold': y_test_2['Gold'].values,
    'Actual Silver': y_test_2['Silver'].values,
    'Actual Bronze': y_test_2['Bronze'].values
})

actual_df_2

,Country,Actual Total,Actual Gold,Actual Silver,Actual Bronze
0,CHN,91.0,40.0,27.0,24.0
1,USA,126.0,40.0,44.0,42.0
2,JPN,45.0,20.0,12.0,13.0
3,AUS,53.0,18.0,19.0,16.0
4,FRA,64.0,16.0,26.0,22.0
...,...,...,...,...,...
201,FSM,0.0,0.0,0.0,0.0
202,NRU,0.0,0.0,0.0,0.0
203,PLW,0.0,0.0,0.0,0.0
204,COL,4.0,0.0,3.0,1.0


And that's it for today. See you in 2028(or 2026 but then you'll have to go up and swap "summer" out for "winter")!